In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# How to use the state evolution package

Let's look at a simple example of how to use the state evolution package with custom teacher-student covariance rmrices. The class has three components:
- `data_model`: this class defines everything concerning the generative model for data - i.e. it initialises the covariances $\Psi, \Phi, \Omega$ and the teacher weights $\theta_{0}$ and pre-computes all the quantities required for the state evolution.
- `model`: this class defines the task. It basically contains the updates for the overlaps and their conjugates. So far, we have implemented ridge and logistic regression.
- `algorithms`: this class defines the iterator for the state evolution.

In [4]:
from state_evolution.models.logistic_regression import LogisticRegression # logistic regression task
from state_evolution.algorithms.state_evolution import StateEvolution # Standard SP iteration

### Example 1: Custom data model: fixed sample complexity

Let's look at a simple example where we input the covariances.

In [5]:
from state_evolution.data_models.custom import Custom # Custom data model. You input the covariances

Recall that the Gaussian covariate model is defined by a teacher-student model with:
- Teacher : $y = f_{0}(\theta_{0}\cdot u)$, $\theta_{0}\sim\mathcal{N}(0,\rm{I}_{p})$
- Student : $\hat{y} = \hat{f}(w\cdot v)$
where $z\in\mathbb{R}^{p}$ and $v\in\mathbb{R}^{d}$ are jointly Gaussian variables with covariances
$$ \Psi = \mathbb{E}uu^{\top}\in\mathbb{R}^{p\times p}, \qquad \Phi = \mathbb{E}uv^{\top}\in\mathbb{R}^{p\times d}, \qquad \Omega = \mathbb{E}vv^{\top}\in\mathbb{R}^{v\times v}
$$.

The class `Custom` takes as input the three covariance matrices that define an instance of the model. 

As an example, let's look at a simple model of a Gaussian teacher $\theta_{0}\sim\mathcal{N}(0,\rm{I}_{p})$ and both the teacher and student are Random Feature models on Gaussian i.i.d. data, with different dimensions and activation functions:
$$
u = \rm{sign}\left(\frac{1}{\sqrt{D}}\bar{\rm{F}}c\right), \qquad v = \rm{erf}\left(\frac{1}{\sqrt{D}}\rm{F}c\right), \qquad c\sim\mathcal{N}(0,\rm{I}_{D})
$$

In this case recall that the covariances can be computed analytically, and are given by:

 \begin{align}
 \Psi = \bar{\kappa}_{1}^2 \bar{\rm{F}}\bar{\rm{F}}^{\top}+\bar{\kappa}_{\star}^2\rm{I}_{p}, && \Phi = \bar{\kappa}_{1}\kappa_{1} \bar{\rm{F}}\rm{F}^{\top}, && \Omega = \kappa_{1}^2 \rm{F}\rm{F}^{\top}+\kappa_{\star}^2\rm{I}_{d}
 \end{align}
 
with $\kappa_{1} \equiv \mathbb{E}\left[\xi\sigma(\xi)\right]$ and $\kappa_{\star}^2 \equiv \mathbb{E}\left[\sigma(\xi)\right]^2-\kappa_{1}^2$ for $\xi\sim\mathcal{N}(0,1)$ (idem for the bar). 

In [46]:
d     = 5000

Psi   = np.identity(d)
Omega = np.identity(d)
Phi   = np.identity(d)

# Teacher weights
theta = np.random.normal(0,1, d)

Now that we have our covariances, we can create our instance of `Custom`:

In [47]:
data_model = Custom(teacher_teacher_cov = Psi, 
                    student_student_cov = Omega, 
                    teacher_student_cov = Phi,
                    teacher_weights = theta)

Now, we need to load our task. Let's look at logistic regression. The `model` class takes as an input the sample complexity $\alpha = n/d$ and the $\ell_2$ regularisation $\lambda>0$

In [17]:
task = LogisticRegression(sample_complexity = 0.5,
                          regularisation= 0.01,
                          data_model = data_model,
                          Delta = 1.0)

All that is left is to initialise the saddle-point equation iterator:

In [18]:
sp = StateEvolution(model = task,
                    initialisation = 'uninformed',
                    tolerance = 1e-7,
                    damping = 0.5,
                    verbose = True,
                    max_steps = 1000)

Now, we can simply iterate it

In [20]:
debut = time()
sp.iterate()
print(f'Elapsed time : {time() - debut}')

t: 0, diff: 477.9685359426921, self overlaps: 0.04213746004115858, teacher-student overlap: 0.037294410668225426
t: 1, diff: 239.9645188711907, self overlaps: 0.13122504413603597, teacher-student overlap: 0.07912260001153269
t: 2, diff: 121.49947692184446, self overlaps: 0.3193662139208878, teacher-student overlap: 0.1326828515594862
t: 3, diff: 62.76933907379919, self overlaps: 0.6528459050781013, teacher-student overlap: 0.19840437552959145
t: 4, diff: 33.642240958693755, self overlaps: 1.1341923301045975, teacher-student overlap: 0.27092766919397904
t: 5, diff: 18.97108187457425, self overlaps: 1.7172626683502519, teacher-student overlap: 0.34253423355424173
t: 6, diff: 11.308440394084455, self overlaps: 2.3432454851880955, teacher-student overlap: 0.40734643805970083
t: 7, diff: 7.098025954448542, self overlaps: 2.9680961468550118, teacher-student overlap: 0.46276839301564693
t: 8, diff: 4.6546663898583205, self overlaps: 3.5652314189552903, teacher-student overlap: 0.5086112962761

Voila, now you can check the result with method `get_info`, which gives everything you might be interested in a dictionary.

In [19]:
sp.get_info()

{'hyperparameters': {'initialisation': 'uninformed',
  'damping': 0.5,
  'max_steps': 1000,
  'tolerance': 1e-07}}

### Example 2: Custom data model: whole learning curve

It is boring to repeat all the pipeline above every time you want to compute a new $\alpha$. Instead, we can encapsulate it in an `experiment` class which allows one to compute a whole learning curve in one go.

In [48]:
from state_evolution.experiments.learning_curve import CustomExperiment

The class `CustomExperiment` takes as argument the task you want (from those implemented), the regularisation and the data_model, apart from all the hyperparameters of the iterator.

In [49]:
my_experiment = CustomExperiment(task = 'logistic_regression', 
                                 regularisation = 1.0, 
                                 data_model = data_model, 
                                 initialisation='uninformed', 
                                 tolerance = 1e-7, 
                                 damping = 0.5, 
                                 verbose = True, 
                                 max_steps = 1000,
                                 sigma = 2.0)

To compute the learning curve, you need to pass a python iterable with the values of the sample complexity you want to compute

In [52]:
my_experiment.learning_curve(alphas = np.linspace(3.0, 15.0, 20))

Runninig sample complexity: 3.0
Runninig sample complexity: 3.6315789473684212
Runninig sample complexity: 4.2631578947368425
Runninig sample complexity: 4.894736842105263
Runninig sample complexity: 5.526315789473684
Runninig sample complexity: 6.157894736842105
Runninig sample complexity: 6.789473684210526
Runninig sample complexity: 7.421052631578947
Runninig sample complexity: 8.052631578947368
Runninig sample complexity: 8.68421052631579
Runninig sample complexity: 9.31578947368421
Runninig sample complexity: 9.94736842105263
Runninig sample complexity: 10.578947368421051
Runninig sample complexity: 11.210526315789473
Runninig sample complexity: 11.842105263157894
Runninig sample complexity: 12.473684210526315
Runninig sample complexity: 13.105263157894736
Runninig sample complexity: 13.736842105263158
Runninig sample complexity: 14.368421052631579
Runninig sample complexity: 15.0


The method `.get_curve()` returns the learning curve as a `pandas.DataFrame`.

In [53]:
my_experiment.get_curve()

,task,gamma,lambda,rho,sample_complexity,V,m,q,test_error,train_loss
0,logistic_regression,1.0,1.0,1.016355,3.000000,0.623050,0.294930,0.299415,0.422639,0.553901
1,logistic_regression,1.0,1.0,1.016355,3.631579,0.576005,0.333504,0.334019,0.417054,0.559249
2,logistic_regression,1.0,1.0,1.016355,4.263158,0.535351,0.367145,0.363274,0.412324,0.563884
3,logistic_regression,1.0,1.0,1.016355,4.894737,0.499901,0.396708,0.388267,0.408253,0.567932
4,logistic_regression,1.0,1.0,1.016355,5.526316,0.468738,0.422866,0.409824,0.404706,0.571492
5,logistic_regression,1.0,1.0,1.016355,6.157895,0.441147,0.446158,0.428577,0.401583,0.574644
6,logistic_regression,1.0,1.0,1.016355,6.789474,0.416556,0.467018,0.445020,0.398810,0.577452
7,logistic_regression,1.0,1.0,1.016355,7.421053,0.394512,0.485797,0.459540,0.396330,0.579967
8,logistic_regression,1.0,1.0,1.016355,8.052632,0.374643,0.502786,0.472445,0.394098,0.582231
9,logistic_regression,1.0,1.0,1.016355,8.684211,0.356648,0.518223,0.483983,0.392078,0.584280


Note you can save it in a csv, you can just call the method `save_experiment`

In [15]:
#my_experiment.save_experiment(name='testing')

### Example 3: defining a model directly as a function of the specta

Note that even though an instance of the Gaussian covariate model is defined by $(\Psi, \Phi, \Omega, \theta_{0})$, the saddle-point equations can be closed on the following scalar quantities:
\begin{align}
\rho = \frac{1}{p}\theta_{0}^{\top}\Psi\theta_{0}, && \omega_{i}\in \rm{spec}(\Omega), && t_{i} = \left(U^{\top}\Phi^{\top}\theta_{0}\theta_{0}^{\top}\Phi U\right)_{ii}, && i=1, \cdots, d
\end{align}
where $\rm{spec}(\Omega)$ are the eigenvalues of $\Omega$ and $U\in\mathbb{R}^{d\times d}$ are the eigenvectors of $\Omega$. 

Therefore, we can also define our `data_model` by directly passing these quantities to the class `CustomSpectra`

In [6]:
from state_evolution.data_models.custom import CustomSpectra

In [7]:
print('Computing the spectrum')
spec_Omega, U = np.linalg.eigh(Omega)

print('Projection in student basis')
t = np.diagonal(U.T @ Phi.T @ theta.reshape(p, 1) @ theta.reshape(1, p) @ Phi @ U)

print('Computing rho')
rho = 1/p * theta.dot(Psi @ theta)

Computing the spectrum
Projection in student basis
Computing rho


Note that $\rho\in\mathbb{R}$, but both $\{\omega_{i}\}_{i=1}^{d}$ and $\{t_{i}\}_{i=1}^{d}$ are $d$-dimensional quantities. Therefore, we will also need to pass $\gamma = p/d$ to our `data_model` in order to run the saddle-point equations.

In [9]:
data_model_spec = CustomSpectra(rho = rho, 
                                spec_Omega = spec_Omega, 
                                diagonal_term = t,
                                gamma = p/d)

In [13]:
my_experiment = CustomExperiment(task = 'logistic_regression', 
                                 regularisation = 0.01, 
                                 data_model = data_model_spec, 
                                 initialisation='uninformed', 
                                 tolerance = 1e-7, 
                                 damping = 0.5, 
                                 verbose = True, 
                                 max_steps = 1000)

my_experiment.learning_curve(alphas = [0.5])

Runninig sample complexity: 0.5
